Dimensionality Reduction cuases some information loss, so its increase in speed does come at the cost of some loss of performance. 

It is also useful for visualization

## PCA

In [3]:
import numpy as np
# Singular Value Decomposition finds unit vectors

X = [] # small 3D dataset
#X_centered = X- X.mean(axis=0)
#U, s, Vt = np.inalg.svd(X_centered)
#c1 = Vt[0]
#c2 = Vt[1]

# To transform dataset, Multiply X by the number of factors you want to keep
# W2 = Vt[:2].T
#X2D = X_centered @ W2

In [6]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
#X2D = pca.fit_transform(X)

# You can see the amount of variation that lies among each principal component
#pca.explained_variance_ratio_

### Choosing the Right Number of Dimensions

In [9]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', as_frame=False)
X_train, y_train = mnist.data[:60_000], mnist.target[:60_000]
X_test, y_test = mnist.data[60_000:], mnist.target[60_000:]

pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

In [18]:
# could then do this:
pca = PCA(n_components=d)
# But you could just do this in the first place
pca = PCA(n_components=0.95)
pca.fit(X_train)
X_reduced = pca.fit_transform(X_train)
print(pca.n_components_, d)

154 154


In [14]:
# Can also use n_components as a hyperparameter
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline

clf = make_pipeline(PCA(random_state=42),
                   RandomForestClassifier(random_state=42))
param_distrib = {
    "pca__n_components": np.arange(10,80),
    "randomforestclassifier__n_estimators": np.arange(50,500)
}
rnd_search = RandomizedSearchCV(clf, param_distrib, n_iter=10, cv=3, random_state=42)
rnd_search.fit(X_train[:1000], y_train[:1000])

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('pca', PCA(random_state=42)),
                                             ('randomforestclassifier',
                                              RandomForestClassifier(random_state=42))]),
                   param_distributions={'pca__n_components': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       6...
       414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426,
       427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439,
       440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452,
       453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465,
       466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478,
       479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491,
       

In [15]:
print(rnd_search.best_params_)

{'randomforestclassifier__n_estimators': 465, 'pca__n_components': 23}


### PCA for Compression
After reducing to components, you can reverse the operation to see the result.

In [19]:
X_recovered = pca.inverse_transform(X_reduced)

### Randomized PCA

In [22]:
rnd_pca = PCA(n_components=154, svd_solver="randomized", random_state=42)
X_reduced = rnd_pca.fit_transform(X_train)

### Incremental PCA
Proceeding implementations required whole training set to fit in memory. Incremental PCA allow splitting training set into mini-batches. Useful for large training sets, or applying PCA online

In [23]:
from sklearn.decomposition import IncrementalPCA

# Split training set into small batches
n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X_train)

In [24]:
# Can use memmap to read the set from disk
filename = "my_mnist.mmap"
X_mmap = np.memmap(filename, dtype="float32", mode="write", shape=X_train.shape)
X_mmap[:] = X_train
X_mmap.flush()

In [26]:
X_mmap = np.memmap(filename, dtype="float32", mode="readonly").reshape(-1, 784)
batch_size = X_mmap.shape[0]
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mmap)

IncrementalPCA(batch_size=60000, n_components=154)

## Random Projection
PCA gets really slow for large datasets. Random Projection may be needed instead.

In [28]:
# The number of dimensions to reduce down to is given by johnson_indenstrauss_min_dim
from sklearn.random_projection import johnson_lindenstrauss_min_dim
m, epsilon = 5_000, 0.1 # epsilon is the max you are willing for the squared distance between 2 points to change
d = johnson_lindenstrauss_min_dim(m, eps=epsilon)

In [30]:
n = 20_000
np.random.seed(42)
P = np.random.randn(d, n) / np.sqrt(d)

X = np.random.randn(m, n)
X_reduced = X @ P.T

In [32]:
from sklearn.random_projection import GaussianRandomProjection

gaussian_rnd_proj = GaussianRandomProjection(eps=epsilon, random_state=42)
X_reduced = gaussian_rnd_proj.fit_transform(X)

Can use SprseRandomProjection to create a sparse random matrix, meaning smaller memory footpring (25 MB vs 1.2 GB in this example). Is faster to generate and transform (50% in this case). If input is sparse, so will output be, unless you specify dence_output=True. It provides comparable quality, so generally it should be prefered.

## LLE
Manifold technique that does not work by projection. Measures how each training instance relates to its nearest neighbor and then looks for low-demiensional represntation that preserves these relationships. Good at unrolling twisted manifolds, particularly when there is not much noise.

Scales poorly to large datasets because one step os O(d * m^2)

In [34]:
from sklearn.datasets import make_swiss_roll
from sklearn.manifold import LocallyLinearEmbedding

X_swiss, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, random_state=42)
X_unrolled = lle.fit_transform(X_swiss)